In [149]:
%load_ext autoreload
%autoreload 2

from torch import empty, cat, arange
from torch.nn.functional import fold, unfold
from torch import nn

from model import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [176]:
torch.set_grad_enabled(False)

In [150]:
# loading image data
noisy_imgs_1, noisy_imgs_2 = torch.load('../../data/train_data.pkl')
train_input, train_target = noisy_imgs_1.float()/255.0, noisy_imgs_2.float()/255.0

In [151]:
# take batch of size 1
input = train_input[0].view(1, 3, 32, 32)
target = train_target[0].view(1, 3, 32, 32)
input.shape

torch.Size([1, 3, 32, 32])

In [152]:
# create some tensors for simple tests
x = torch.randn((1, 3, 4, 4))

# y = torch.ones(x.shape)
y = torch.ones((1, 4, 3, 3))

### Testing our Conv2d Layer 

Forward pass

In [156]:
input.shape, target.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 3, 32, 32]))

In [188]:
input = train_input.narrow(0, 100, 100)
input.shape

torch.Size([100, 3, 32, 32])

In [189]:
# testing our conv2d layer
my_conv = Conv2d(3, 4, 2)
output = my_conv.forward(input)
output.shape

torch.Size([100, 4, 31, 31])

In [190]:
torch.testing.assert_allclose(my_conv.forward(input), torch.nn.functional.conv2d(input, my_conv.weight, my_conv.bias))

In [184]:
criterion = MSE()

loss = criterion.forward(output, my_conv.forward(target))
#loss.requires_grad_()
dl = criterion.backward()

Backward pass

In [186]:
back = my_conv.backward(dl)
back.shape

torch.Size([1, 3, 32, 32])

### Testing our Upsampling (TransposeConv2d) layer

Forward pass

In [165]:
my_t_conv = Upsampling(3, 5, 2, stride=2)
upsampled = my_t_conv.forward(input)
upsampled.shape

torch.Size([1, 5, 64, 64])

In [166]:
torch.testing.assert_allclose(my_t_conv.forward(input), torch.nn.functional.conv_transpose2d(input, my_t_conv.weight, my_t_conv.bias, stride=2))

Backward pass

In [168]:
back_t = my_t_conv.backward(upsampled)
back_t

tensor([[[[-0.0632, -0.0536, -0.0469,  ...,  0.1634,  0.1538,  0.1692],
          [-0.0580, -0.0528, -0.0511,  ...,  0.1688,  0.1671,  0.1682],
          [-0.0542, -0.0628, -0.0577,  ...,  0.1619,  0.1682,  0.1631],
          ...,
          [-0.2246, -0.2358, -0.0902,  ..., -0.0769, -0.0654, -0.0685],
          [-0.1425, -0.1096, -0.2106,  ..., -0.0803, -0.1245, -0.0807],
          [-0.1841, -0.1282, -0.0213,  ..., -0.0759, -0.0730, -0.0999]],

         [[ 0.3516,  0.3632,  0.3557,  ...,  0.4363,  0.4655,  0.4298],
          [ 0.3534,  0.3553,  0.3559,  ...,  0.4327,  0.4321,  0.4341],
          [ 0.3534,  0.3556,  0.3522,  ...,  0.4302,  0.4341,  0.4370],
          ...,
          [ 0.2978,  0.3101,  0.3491,  ...,  0.3453,  0.3475,  0.3497],
          [ 0.3318,  0.3479,  0.3237,  ...,  0.3441,  0.3300,  0.3454],
          [ 0.3200,  0.3504,  0.4012,  ...,  0.3410,  0.3464,  0.3333]],

         [[ 0.0650,  0.0720,  0.0877,  ...,  0.3209,  0.2190,  0.3213],
          [ 0.0710,  0.0769,  

### Sequential model testing

In [171]:
model = Sequential(Conv2d(3,4,2), ReLU())
forward_pass = model.forward(x)
#forward_pass
backward_pass = model.backward(forward_pass)


### Testing model

In [200]:
train_input = train_input[:50, :, :, :]
train_target = train_target[:50, :, :, :]

In [201]:
model = Model()
model.train(train_input, train_target, 5)

RuntimeError: start (0) + length (10) exceeds dimension size (1).